In [3]:
# install openai 1.10
#!pip install openai==1.10.0

In [30]:
import openai
import json
import ast

# api model settings

In [34]:
OPENAI_API_KEY=""
OPENAI_API_VERSION=""
OPENAI_API_ENDPOINT=""

In [26]:
client = openai.AzureOpenAI(
    api_key = OPENAI_API_KEY,
    api_version = OPENAI_API_VERSION,
    azure_endpoint = OPENAI_API_ENDPOINT,
)

In [31]:
# defailed documentation: https://platform.openai.com/docs/api-reference/chat/create?lang=python

def send_prompt(prompt, model = "gpt-35-turbo", is_json=False):
    if is_json:
        response = client.chat.completions.create(
            model= model, # model = "deployment_name".
            response_format={ "type": "json_object" },
            messages=[{"role": "user", "content": prompt}]
        )
    else:
        response = client.chat.completions.create(
            model= model, # model = "deployment_name".
            messages=[{"role": "user", "content": prompt}]
        )

    resp =response.choices[0].message.content
    return resp


def send_messages(messages, model = "gpt-35-turbo", is_json=False):
    if is_json:
        response = client.chat.completions.create(
            model= model, 
            response_format={ "type": "json_object" },
            messages=messages
        )
    else:
        response = client.chat.completions.create(
            model= model, 
            messages=messages
        )

    resp =response.choices[0].message.content
    return resp

In [36]:
send_messages(msg)

'I am an AI language model created by OpenAI called GPT-3. I have been trained on a wide variety of texts and can assist with answering questions, providing information, and engaging in conversation. How can I assist you today?'

# input from frontend: transcript + notes

In [40]:
transcript = [
    {"person":"Therapist", "timestamp":"xxx", "content":"Good afternoon. I'm Dr. Smith, your therapist. How are you feeling today?"},
    {"person":"Patient", "timestamp":"xxx", "content":"Hi, Dr. Smith. I'm doing okay, I guess. A bit anxious, but that's normal, right?"},
        {"person":"Therapist", "timestamp":"xxx", "content":"Absolutely. It's completely normal to feel a bit anxious, especially when starting therapy. To get started, could you tell me a bit about yourself? Your age, occupation, and any relevant personal background?"},
    {"person":"Patient", "timestamp":"xxx", "content":"Sure, I'm 28 years old, work in marketing, and recently moved to the city. The new job and environment have been a bit overwhelming."},
        {"person":"Therapist", "timestamp":"xxx", "content":"It sounds like you've been going through significant changes. Can you share what prompted you to seek therapy?"},
    {"person":"Patient", "timestamp":"xxx", "content":"Well, the move and new job have been stressful, but it's more than that. I've been feeling unusually sad and having trouble sleeping. My friend suggested it might be depression, so I thought I should talk to someone."},
        {"person":"Therapist", "timestamp":"xxx", "content":"Thank you for sharing that. It's commendable that you reached out for support. Can you tell me more about your mood and sleep patterns? Have you noticed any specific triggers for these feelings?"},
    {"person":"Patient", "timestamp":"xxx", "content":"My mood has been consistently low, and I often feel overwhelmed. I've been sleeping poorly, waking up in the middle of the night and struggling to get back to sleep. It's affecting my energy levels during the day."},
        {"person":"Therapist", "timestamp":"xxx", "content":"I see. Have you experienced any significant life events or changes recently that might be contributing to these feelings?"},
    {"person":"Patient", "timestamp":"xxx", "content":"Well, besides the move and job, there was a breakup about six months ago that I haven't fully processed. It was a long-term relationship, and I think it still weighs on me."},
        {"person":"Therapist", "timestamp":"xxx", "content":"Breakups can be challenging, and it's understandable that it might still be affecting you. In addition to the low mood and sleep difficulties, have you noticed any changes in your appetite, concentration, or interest in activities you used to enjoy?"},
    {"person":"Patient", "timestamp":"xxx", "content":"Yeah, my appetite has decreased, and I've lost some weight unintentionally. Concentration has been tough at work, and I don't really feel like doing things I used to enjoy."},
        {"person":"Therapist", "timestamp":"xxx", "content":"Thank you for sharing those details. It helps me understand your situation better. Let's talk about your support system. Do you have friends or family you can rely on for emotional support?"},
    {"person":"Patient", "timestamp":"xxx", "content":"I have a few friends in the city, but they're busy with their own lives. My family is back home, and we talk occasionally, but I don't want to burden them."},
        {"person":"Therapist", "timestamp":"xxx", "content":"It's essential to have a support system, but it's okay to seek professional help too. Moving forward, we can explore strategies to cope with the changes and work on improving your overall well-being. How does that sound to you?"},
    {"person":"Patient", "timestamp":"xxx", "content":"Yeah, I'm willing to try. I just want to feel better."},
        {"person":"Therapist", "timestamp":"xxx", "content":"That's a great attitude. In our next sessions, we'll delve deeper into your experiences and work on developing coping mechanisms. Is there anything else you'd like to share before we conclude today's session?"},
    {"person":"Patient", "timestamp":"xxx", "content":"I think that covers it for now. Thanks for listening, Dr. Smith."},
        {"person":"Therapist", "timestamp":"xxx", "content":"You're welcome. I appreciate your openness. We'll take things one step at a time. I look forward to our continued work together."},
]

In [42]:
notes =  ["job and environment", "sleeping issues"]

# prompt to extract information

In [61]:
def get_prompt_documentation(transcript, notes):
    return """Generete a structured documentation by extracting the information from the transcript and return in structured format. 
    If there are some part of information that are not talked in the transcirpt, return with "TBD". This is the transcription: 
    {{{transcript}}}. This is the doctor's notes: {{{notes}}}. Return your response in following JSON format.
    
    Desired format:
    {{
        "Assessment And Diagnoisis": {{
            "Formal Diagnosis": [LIST OF STRING],
            "Identified Challenges": <STRING>,
            "Identified Strengths": <STRING>，
        }},
        
        "Treatment And Plans": {{
            "Short-term and Long-term Goals": {{ "Long-term Goals": <STRING>,  "Short-term Goals": <STRING>}},
            "Therapeutic Approaches and Adjustments": <LIST OF STRING>,
            "Frequency and Duration of Sessions": {{"Frequency": <STRING>, "Duration":<STRING>}},
            "Progression of Treatment Over Time": <STRING>,
        }},
        
        "Medication Management": {{
            "Prescribed Medications": <LIST OF STRING>,
            "Effects and Potential Side Effects": <LIST OF STRING>,
        }},
        
        "Crisis Management": {{
            "Plan for Crises or Emergencies": <LIST OF STRING>,
            "Emergency Contact Information": {{"Therapist":<STRING>, "Local Crisis Hotline":<STRING>}}
        }},
        
        "Self Care Stategies": {{
            "Recommendations for Self-Care Activities": <LIST OF STRING>,
            "Coping Mechanisms": <LIST OF STRING>,
            "Strategies to Manage Challenges": <LIST OF STRING>
        }}
    }}
    """.format(transcript=transcript, notes=notes)

In [38]:
def get_prompt_patient_background(transcript, notes):
    return """Generete a structured documentation by extracting the information from the transcript and return in structured format. If there are some part of information that are not talked in the transcirpt, return with "TBD". This is the transcription: {{{transcript}}}. This is the doctor's notes: {{{notes}}}. Return your response in following JSON format.
    
    Desired format:
    {{
        "Patient Name": <STRING>,
        "Demographics": {{ "Age":<INT>, "Gender":<STRING>, "Occupation":<STRING>, "Contact Information": <STRING>}},
        "Relevant Personal History": [LIST OF STRING],
        "Presenting Issues and Concerns": [LIST OF STRING],
    }}
    """.format(transcript=transcript, notes=notes)


def get_prompt_assessment_and_diagnoisis(transcript, notes):
    return """Generete a structured documentation by extracting the information from the transcript and return in structured format. If there are some part of information that are not talked in the transcirpt, return with "TBD". This is the transcription: {{{transcript}}}. This is the doctor's notes: {{{notes}}}. Return your response in following JSON format.
    
    Desired format:
    {{
        "Formal Diagnosis": [LIST OF STRING],
        "Identified Challenges": <STRING>,
        "Identified Strengths": <STRING>
    }}
    """.format(transcript=transcript, notes=notes)


def get_prompt_treatment_and_plans(transcript, notes):
    return """Generete a structured documentation by extracting the information from the transcript and return in structured format. 
    If there are some part of information that are not talked in the transcirpt, return with "TBD". This is the transcription: 
    {{{transcript}}}. This is the doctor's notes: {{{notes}}}. Return your response in following JSON format.
    
    Desired format:
    {{
        "Short-term and Long-term Goals": {{ "Long-term Goals": <STRING>,  "Short-term Goals": <STRING>}},
        "Therapeutic Approaches and Adjustments": <LIST OF STRING>,
        "Frequency and Duration of Sessions": {{"Frequency": <STRING>, "Duration":<STRING>}},
        "Progression of Treatment Over Time": <STRING>,
    }}
    """.format(transcript=transcript, notes=notes)

def get_prompt_medication_management(transcript, notes):
    return """Generete a structured documentation by extracting the information from the transcript and return in structured format. If there are some part of information that are not talked in the transcirpt, return with "TBD". This is the transcription: {{{transcript}}}. This is the doctor's notes: {{{notes}}}. Return your response in following JSON format.
    
    Desired format:
    {{
        "Prescribed Medications": <LIST OF STRING>,
        "Effects and Potential Side Effects": <LIST OF STRING>,
    }}
    """.format(transcript=transcript, notes=notes)


def get_prompt_crisis_management(transcript, notes):
    return """Generete a structured documentation by extracting the information from the transcript and return in structured format. If there are some part of information that are not talked in the transcirpt, return with "TBD". This is the transcription: {{{transcript}}}. This is the doctor's notes: {{{notes}}}. Return your response in following JSON format.
    
    Desired format:
    {{
        "Plan for Crises or Emergencies": <LIST OF STRING>,
        "Emergency Contact Information": {{"Therapist":<STRING>, "Local Crisis Hotline":<STRING>}}
    }}
    """.format(transcript=transcript, notes=notes)

def get_prompt_self_care_strategies(transcript, notes):
    return """Generete a structured documentation by extracting the information from the transcript and return in structured format. If there are some part of information that are not talked in the transcirpt, return with "TBD". This is the transcription: {{{transcript}}}. This is the doctor's notes: {{{notes}}}. Return your response in following JSON format.
    
    Desired format:
    {{
        "Recommendations for Self-Care Activities": <LIST OF STRING>,
        "Coping Mechanisms": <LIST OF STRING>,
        "Strategies to Manage Challenges": <LIST OF STRING>
    }}
    """.format(transcript=transcript, notes=notes)


## testing prompt

In [50]:
# parse string into python dictionary
def parse_to_json(resp):
    return json.loads(resp)
                
def parse_ask_character_to_json(resp):
    try:
        return json.loads(resp)
    except Exception as e:  # incase it the dict keys is wrapped by ' instead of "
        return ast.literal_eval(resp)

In [62]:
settings = "You are an assistant agent to help mental health therapist to write their documentation of the therapy. You need to help to extract information from the audio transcript. Do not give your analysis report."
messages = [
    {"role":"system", "content":settings},
    {"role":"user", "content":get_prompt_documentation(transcript, notes)}
]

In [63]:
# input length. Make sure it does not excceed the limit
len(str(messages))

6243

In [64]:
resp = send_messages(messages)
resp

'{\n    "Assessment And Diagnoisis": {\n        "Formal Diagnosis": ["TBD"],\n        "Identified Challenges": "job and environment, sleeping issues",\n        "Identified Strengths": "TBD"\n    },\n    \n    "Treatment And Plans": {\n        "Short-term and Long-term Goals": { "Long-term Goals": "TBD",  "Short-term Goals": "TBD"},\n        "Therapeutic Approaches and Adjustments": ["TBD"],\n        "Frequency and Duration of Sessions": {"Frequency": "TBD", "Duration": "TBD"},\n        "Progression of Treatment Over Time": "TBD"\n    },\n    \n    "Medication Management": {\n        "Prescribed Medications": ["TBD"],\n        "Effects and Potential Side Effects": ["TBD"]\n    },\n    \n    "Crisis Management": {\n        "Plan for Crises or Emergencies": ["TBD"],\n        "Emergency Contact Information": {"Therapist":"TBD", "Local Crisis Hotline":"TBD"}\n    },\n    \n    "Self Care Stategies": {\n        "Recommendations for Self-Care Activities": ["TBD"],\n        "Coping Mechanisms"

In [65]:
res = parse_to_json(resp)
res

{'Assessment And Diagnoisis': {'Formal Diagnosis': ['TBD'],
  'Identified Challenges': 'job and environment, sleeping issues',
  'Identified Strengths': 'TBD'},
 'Treatment And Plans': {'Short-term and Long-term Goals': {'Long-term Goals': 'TBD',
   'Short-term Goals': 'TBD'},
  'Therapeutic Approaches and Adjustments': ['TBD'],
  'Frequency and Duration of Sessions': {'Frequency': 'TBD',
   'Duration': 'TBD'},
  'Progression of Treatment Over Time': 'TBD'},
 'Medication Management': {'Prescribed Medications': ['TBD'],
  'Effects and Potential Side Effects': ['TBD']},
 'Crisis Management': {'Plan for Crises or Emergencies': ['TBD'],
  'Emergency Contact Information': {'Therapist': 'TBD',
   'Local Crisis Hotline': 'TBD'}},
 'Self Care Stategies': {'Recommendations for Self-Care Activities': ['TBD'],
  'Coping Mechanisms': ['TBD'],
  'Strategies to Manage Challenges': ['TBD']}}